## 5.7 パネルデータ分析(1) データ構築
これは以下の内容を参考にしています。

https://bashtage.github.io/linearmodels/panel/examples/examples.html

まず、パネルデータのセッティングを行います。linearmodels.datasetsの中のwage_panelデータを読み込み、dataに代入します。

続いて、Pandasのカテゴリデータとして、dataからyearを取り出し、それをyearに代入します。

続いて、dataにインデックス値としてnrおよびyearを設定します。

続いて、カテゴリ化し変数yearに代入した年次データを、再度data内のyearに代入します。

最後に、データの概要をprintを用いて標準出力します。

In [1]:
import pandas as pd
from linearmodels.datasets import wage_panel

#データセットを読み込む
data = wage_panel.load()
#年情報データを取得する
year = pd.Categorical(data.year)
#インデックス情報を指定する
# nr が個人の識別子
# year が年の識別情報
data = data.set_index(["nr", "year"])
data["year"] = year

#データの概要を抽出する
print(wage_panel.DESCR)


F. Vella and M. Verbeek (1998), "Whose Wages Do Unions Raise? A Dynamic Model
of Unionism and Wage Rate Determination for Young Men," Journal of Applied
Econometrics 13, 163-183.

nr                       person identifier
year                     1980 to 1987
black                    =1 if black
exper                    labor market experience
hisp                     =1 if Hispanic
hours                    annual hours worked
married                  =1 if married
educ                     years of schooling
union                    =1 if in union
lwage                    log(wage)
expersq                  exper^2
occupation               Occupation code



データセットの概要が出力されます。今回のデータセットは、F. Vella and M. Verbeek (1998)が、Journal of Applied Econometricsに公開した論文に基づくことが確認できる。

そして、データセットの一部を出力する。

In [2]:
#データの中身を確認する
print(data.head())

         black  exper  hisp  hours  married  educ  union     lwage  expersq  \
nr year                                                                       
13 1980      0      1     0   2672        0    14      0  1.197540        1   
   1981      0      2     0   2320        0    14      1  1.853060        4   
   1982      0      3     0   2940        0    14      0  1.344462        9   
   1983      0      4     0   2960        0    14      0  1.433213       16   
   1984      0      5     0   3071        0    14      0  1.568125       25   

         occupation  year  
nr year                    
13 1980           9  1980  
   1981           9  1981  
   1982           9  1982  
   1983           9  1983  
   1984           5  1984  


個人の識別子であるnrごとに、年ごとのパラメータが複数収められたパネルデータであることが確認できる。

続いて、データの概要について確認する。describe()で最大値や最小値、平均値などが確認できるのは、クロスセクションなデータと同じです。

In [3]:
#データの概要を確認する
print(data.describe())

             black        exper         hisp        hours      married  \
count  4360.000000  4360.000000  4360.000000  4360.000000  4360.000000   
mean      0.115596     6.514679     0.155963  2191.257339     0.438991   
std       0.319777     2.825873     0.362862   566.352301     0.496321   
min       0.000000     0.000000     0.000000   120.000000     0.000000   
25%       0.000000     4.000000     0.000000  2040.000000     0.000000   
50%       0.000000     6.000000     0.000000  2080.000000     0.000000   
75%       0.000000     9.000000     0.000000  2414.250000     1.000000   
max       1.000000    18.000000     1.000000  4992.000000     1.000000   

              educ        union        lwage      expersq   occupation  
count  4360.000000  4360.000000  4360.000000  4360.000000  4360.000000  
mean     11.766972     0.244037     1.649147    50.424771     4.988532  
std       1.746181     0.429564     0.532609    40.781991     2.319978  
min       3.000000     0.000000    -3.579

## 5.8 パネルデータ分析(2) Pooled OLS
それぞれのエントリ（今回のデータの場合はnr）による特性を考慮しない回帰モデルとしての、PooledOLSを実行する。

ここでは、linearmodelsのPooledOLSをインポートします。今回は、被説明変数としてlwage、説明変数に、expersq、union、marriedなどの変数や、year（年ダミー）とし、これまでの回帰分析と同じく、sm.add_constantを用いて定数項を加えて回帰分析を行う。

In [4]:
#必要なパッケージをインポートする
import statsmodels.api as sm
from linearmodels.panel import PooledOLS

#説明変数を指定する
#exog_vars = ["black", "hisp", "exper", "expersq", "married", "educ", "union", "year"]
exog_vars = ["expersq", "union", "married", "year"]
exog = sm.add_constant(data[exog_vars])
#被説明変数には data.lwage を指定する
mod = PooledOLS(data.lwage, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                        0.1246
Estimator:                  PooledOLS   R-squared (Between):              0.0902
No. Observations:                4360   R-squared (Within):               0.1646
Date:                Tue, Nov 15 2022   R-squared (Overall):              0.1246
Time:                        11:46:56   Log-likelihood                   -3149.2
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      61.920
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,4349)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             61.920
                            

プラスに作用する変数(const, union, married)、マイナスに作用する変数(expersq)があることが確認できる。係数ごとにt値、P値が出力されるのは、これまでの線形回帰とそれほど変わらない。違うポイントは、Time Periodsが表示されている点です。今回のデータセットは8時点間に基づくパネルデータであることが確認できる。(1980-1987の8時点）

## 5.9 パネルデータ分析(3) 固定効果モデル
固定効果モデル(fixed-effect model)によりパネルデータの分析を行う。

固定効果モデルでは、エントリーごとの固有の特性が存在するとして分析を行う。例えば、従業員ごとの特性や企業ごとの特性をコントロールしたい場合、固定効果モデルに基づきパネルデータの分析を行う。

linearmodelsからPanelOLSをインポートします。説明変数のリストを指定し、PanelOLSに（被説明変数、説明変数）の順に値を指定し、fit()メソッドを用いてモデルを実行する。

In [5]:
#必要なパッケージをインポートする
from linearmodels.panel import PanelOLS

#説明変数を指定する
exog_vars = ["expersq", "union", "married", "year"]
exog = sm.add_constant(data[exog_vars])
#被説明変数には data.lwage を指定する
#Fixed Effect で分析するために、entity_effect=Trueを指定する
mod = PanelOLS(data.lwage, exog, entity_effects=True)
fe_res = mod.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                        0.1806
Estimator:                   PanelOLS   R-squared (Between):             -0.0052
No. Observations:                4360   R-squared (Within):               0.1806
Date:                Tue, Nov 15 2022   R-squared (Overall):              0.0807
Time:                        11:49:15   Log-likelihood                   -1324.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      83.851
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,3805)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             83.851
                            

上記結果からは、若干、係数がPooledOLSと異なっていること、Estimaator:の部分がPanelOLSになっていること、出力の部分に"Included effects: Entity"と明記されていることなどが確認できる。

## 5.10 パネルデータ分析(4) 変動効果モデル
変動効果モデル(Random Effect Model)では、エントリごとの固有の効果を確率的な要因として捉え分析を行う。

パッケージRandomEffectをlinearmodelsからインポートし、これまでと同じく、被説明変数と説明変数からパネルデータに指定します。

In [6]:
#必要なパッケージをインポートする
from linearmodels.panel import RandomEffects

#説明変数を指定する
exog_vars = ["expersq", "union", "married", "year"]
exog = sm.add_constant(data[exog_vars])
#被説明変数には data.lwage を指定する
mod = RandomEffects(data.lwage, exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                  lwage   R-squared:                        0.1643
Estimator:              RandomEffects   R-squared (Between):              0.0564
No. Observations:                4360   R-squared (Within):               0.1780
Date:                Tue, Nov 15 2022   R-squared (Overall):              0.1127
Time:                        11:51:25   Log-likelihood                   -1629.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      85.492
Entities:                         545   P-value                           0.0000
Avg Obs:                       8.0000   Distribution:                 F(10,4349)
Min Obs:                       8.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             85.492
                            

PooledOLSやFixed Effect Modelに比べて変数の係数や決定係数に違いがあることが確認できます。一般的に、係数はPooledOLSとFixed Effect Modelの中間の値になることが知られている。

## 5.11 パネルデータ分析(5) モデルの比較
最後に、PooledOLSモデル、Fixed Effect モデル、Random Effectモデルの係数を一覧表示します。

In [7]:
from linearmodels.panel import compare

print(compare({"FE": fe_res, "RE": re_res, "Pooled": pooled_res}))

                            Model Comparison                           
                                    FE                RE         Pooled
-----------------------------------------------------------------------
Dep. Variable                    lwage             lwage          lwage
Estimator                     PanelOLS     RandomEffects      PooledOLS
No. Observations                  4360              4360           4360
Cov. Est.                   Unadjusted        Unadjusted     Unadjusted
R-squared                       0.1806            0.1643         0.1246
R-Squared (Within)              0.1806            0.1780         0.1646
R-Squared (Between)            -0.0052            0.0564         0.0902
R-Squared (Overall)             0.0807            0.1127         0.1246
F-statistic                     83.851            85.492         61.920
P-value (F-stat)                0.0000            0.0000         0.0000
=====================     ============   ===============   =====